# Data Exploration

In [2]:
import beatthebookies.data as btd
import beatthebookies.trainer as btt
from beatthebookies.trainer import Trainer

In [3]:
import mlflow
import time
import pandas as pd
# import warnings
from beatthebookies.data import get_data, get_betting_data
from beatthebookies.utils import simple_time_tracker, compute_scores, compute_overall_scores

from mlflow.tracking import MlflowClient
from memoized_property import memoized_property
from sklearn.linear_model import LogisticRegression, Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
#from beatthebookies.encoders import CustomNormaliser, CustomStandardScaler
from tempfile import mkdtemp
from beatthebookies.bettingstrategy import compute_profit

In [4]:
experiment = "BeatTheBookies"
params = dict(season='2015/2016',
              full=False,
              upload=True,
              local=False,  # set to False to get data from GCP (Storage or BigQuery)
              gridsearch=False,
              optimize=False,
              estimator="KNNClassifier",
              mlflow=True,  # set to True to log params to mlflow
              experiment_name=experiment,
              pipeline_memory=None,
              feateng=None,
              n_jobs=-1)
df = get_data(**params)
betting_data = get_betting_data(**params)
bet = 10
df.dropna(inplace=True)
print(df.shape)
X = df.drop(columns=['id', 'season', 'date', 'stage', 'home_team_goal', 'away_team_goal', 'home_team', 'away_team', 'home_w', 'away_w', 'draw'])
y = df[['home_w', 'away_w', 'draw']]
t = Trainer(X=X, y=y, **params)
t.train()
t.evaluate(bet)

get_data 8.9
get_betting_data 0.05
(380, 47)
train 0.48


/Users/nico/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/nico/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


TypeError: evaluate() missing 1 required positional argument: 'bet'

In [11]:
d = {'col1': [1, 2], 'col2': [3, 4]}
p = {'col6': [1, 5], 'col9': [2, 6]}

In [12]:
p = pd.DataFrame(data=d)
k = pd.DataFrame(data=p)

In [15]:
p

,col1,col2
0,1,3
1,2,4


In [16]:
k

,col1,col2
0,1,3
1,2,4


In [13]:
pd.np.multiply(p,k)

,col1,col2
0,1,9
1,4,16


In [9]:
df.head(250)

,id,season,stage,date,home_team,away_team,home_team_goal,away_team_goal,home_w,away_w,...,da_x,dtw_x,bups_y,bupp_y,ccp_y,ccc_y,ccs_y,dp_y,da_y,dtw_y
0,4390,2015/2016,1,2015-08-08 00:00:00,Bournemouth,Aston Villa,0,1,0,1,...,50,62,63,54,60,48,38,35,44,54
1,4391,2015/2016,1,2015-08-08 00:00:00,Chelsea,Swansea City,2,2,0,0,...,41,46,45,42,34,36,55,31,47,42
2,4392,2015/2016,1,2015-08-08 00:00:00,Everton,Watford,2,2,0,0,...,58,59,61,52,54,42,45,38,52,45
3,4393,2015/2016,1,2015-08-08 00:00:00,Leicester City,Sunderland,4,2,1,0,...,65,55,43,51,59,50,55,47,45,49
4,4394,2015/2016,1,2015-08-08 00:00:00,Manchester United,Tottenham Hotspur,1,0,1,0,...,53,56,47,40,41,41,63,63,54,56
5,4396,2015/2016,1,2015-08-08 00:00:00,Norwich City,Crystal Palace,1,3,0,1,...,39,51,61,67,67,69,50,36,34,41
6,4389,2015/2016,1,2015-08-09 00:00:00,Arsenal,West Ham United,0,2,0,1,...,44,52,77,57,68,73,29,30,37,41
7,4395,2015/2016,1,2015-08-09 00:00:00,Newcastle United,Southampton,2,2,0,0,...,39,51,52,46,42,54,30,50,65,44
8,4397,2015/2016,1,2015-08-09 00:00:00,Stoke City,Liverpool,0,1,0,1,...,37,51,66,45,34,34,46,51,52,61
9,4398,2015/2016,1,2015-08-10 00:00:00,West Bromwich Albion,Manchester City,0,3,0,1,...,40,49,59,29,38,36,24,48,47,54


In [8]:
df['profit'] = -10
df.loc[(y_pred == y_true), 'profit'] = (((df['WHH']*df['home_w'])+(df['WHD']*df['draw'])+(df['WHA']*df['away_w'])) * 10) - 10
btb_profit_total = df['profit'].sum()
btb_profit = df['profit'].tail(1)

NameError: name 'y_pred' is not defined

In [ ]:
import sqlite3
import pandas as pd

path = "../data/"  #Insert path here
database = path + "database.sqlite"
conn = sqlite3.connect(database)

cursor = conn.cursor()
print(cursor.fetchall())

matches_df = pd.read_sql(
    '''
    SELECT id,league_id,season,stage,home_team_api_id,away_team_api_id,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
    FROM Match m
    WHERE league_id == 1729
    AND season == '2015/2016'
    ORDER BY stage ASC
    ''', conn)

matches_df.head(100)

In [ ]:
#matches_df['home_goals']= matches_df.groupby('home_team_api_id')['home_team_goal'].cumsum().fillna(0)

In [ ]:
#matches_df['away_goals']= matches_df.groupby('home_team_api_id')['away_team_goal'].cumsum().fillna(0)

In [ ]:
#matches_df.head(3)

In [ ]:
matches_df['shoton'][0]

In [ ]:
from xml.etree.ElementTree import XML, fromstring
myxml = fromstring(matches_df['shoton'][0])

### Variables to consider

1. Home Team Total Wins	
2. Home Team Home wins	
3. Home Team Total Losses	
4. Home Team Home losses	
5. Away Team Rank	
6. Away Team Total Wins	
7. Away Team Away wins	
8. Away Team Total Losses	
9. Away Team Away Losses	
10. Home Team Total Goals	
11. Home Team Home Goals	
12. Home Team Total Goals allowed	
13. Home Team Home Goals Allowed	
14. Away Team Total Goals	
15. Away Team Away Goals	
16. Away Team Total Goals Allowed	
17. Away Team Away Goals Allowed	
18. Home Team Win y/n

### home / draw / away

In [ ]:
matches_df['home']=0
matches_df['away']=0
matches_df['home_draw']=0
#matches_df['away_draw']=0

matches_df.loc[matches_df['home_team_goal'] > matches_df['away_team_goal'],'home']=1
matches_df.loc[matches_df['home_team_goal'] < matches_df['away_team_goal'],'away']=1
matches_df.loc[matches_df['home_team_goal'] == matches_df['away_team_goal'],'home_draw']=1
#matches_df.loc[matches_df['home_team_goal'] == matches_df['away_team_goal'],'away_draw']=1

In [ ]:
matches_df.head(5)

In [ ]:
matches_df['lowest_odds']=0
matches_df['highest_odds']=0
matches_df['draw_only']=0
matches_df['home_only']=0
matches_df['away_only']=0

In [ ]:
matches_df.head(5)

In [ ]:
matches_df['min_profit']=-10

### Lowest Odds

In [ ]:
matches_df.loc[(matches_df[['LBH','LBD','LBA']].min(axis=1) == matches_df['LBH']) & (matches_df['home'] == 1), 'min_profit'] = matches_df['LBH']*10 
matches_df.loc[(matches_df[['LBH','LBD','LBA']].min(axis=1) == matches_df['LBA']) & (matches_df['away'] == 1), 'min_profit'] = matches_df['LBA']*10 
matches_df.loc[(matches_df[['LBH','LBD','LBA']].min(axis=1) == matches_df['LBD']) & (matches_df['home_draw'] == 1), 'min_profit'] = matches_df['LBD']*10 

In [ ]:
matches_df['min_profit_cum']= matches_df['min_profit'].cumsum().fillna(0)

In [ ]:
matches_df.tail(10)

### Highest Odds

In [ ]:
matches_df['max_profit']=-10

In [ ]:
matches_df.loc[(matches_df[['LBH','LBD','LBA']].max(axis=1) == matches_df['LBH']) & (matches_df['home'] == 1), 'max_profit'] = matches_df['LBH']*10
matches_df.loc[(matches_df[['LBH','LBD','LBA']].max(axis=1) == matches_df['LBA']) & (matches_df['away'] == 1), 'max_profit'] = matches_df['LBA']*10
matches_df.loc[(matches_df[['LBH','LBD','LBA']].max(axis=1) == matches_df['LBD']) & (matches_df['home_draw'] == 1), 'max_profit'] = matches_df['LBD']*10

In [ ]:
matches_df['max_profit_cum']= matches_df['max_profit'].cumsum().fillna(0)

In [ ]:
matches_df.head(200)

### Draw / Home / Away Only

In [ ]:
matches_df['draw_profit']=-10

In [ ]:
matches_df.loc[(matches_df['home_draw'] == 1), 'draw_profit'] = matches_df['LBD']*10

In [ ]:
matches_df['draw_profit_cum']= matches_df['draw_profit'].cumsum().fillna(0)

In [ ]:
matches_df['home_profit']=-10

In [ ]:
matches_df.loc[(matches_df['home'] == 1), 'home_profit'] = matches_df['LBH']*10

In [ ]:
matches_df['home_profit_cum']= matches_df['home_profit'].cumsum().fillna(0)

In [ ]:
matches_df['away_profit']=-10

In [ ]:
matches_df.loc[(matches_df['away'] == 1), 'away_profit'] = matches_df['LBA']*10

In [ ]:
matches_df['away_profit_cum']= matches_df['away_profit'].cumsum().fillna(0)

In [ ]:
matches_df.tail(5)

In [ ]:
"""matches_df['home_wins_cum']=matches_df.groupby('home_team_api_id')['home'].cumsum()
matches_df['away_wins_cum']=matches_df.groupby('home_team_api_id')['away'].cumsum()
matches_df['home_draw_cum']=matches_df.groupby('home_team_api_id')['home_draw'].cumsum()
#matches_df['away_draw_cum']=matches_df.groupby('away_team_api_id')['away_draw'].cumsum()
"""

In [ ]:
"""matches_df.groupby('stage').head(20)"""